In [29]:
from audiocraft.models import MusicGen

In [30]:
default_model_checkpoint = 'facebook/musicgen-small'

In [31]:
model = MusicGen.get_pretrained(default_model_checkpoint)

ConfigAttributeError: Missing key memory_saver
    full_key: memory_saver
    object_type=dict

In [1]:
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [2]:
from audiocraft.models.loaders import _get_state_dict, _delete_param, load_compression_model, load_lm_model_ckpt

In [3]:
import torch
from omegaconf import OmegaConf, DictConfig
from audiocraft.models import builders

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [6]:
lm_trained_checkpoint = 'C:/tmp/audiocraft_MusicGen/xps/d56f6387/checkpoint.th'
default_model_checkpoint = 'facebook/musicgen-small'

In [7]:
pkg = _get_state_dict(lm_trained_checkpoint)
cfg = OmegaConf.create(pkg['xp.cfg'])
cfg.device = str(device)
if cfg.device == 'cpu':
    cfg.dtype = 'float32'
else:
    cfg.dtype = 'float16'
cache_dir=None
pkg_def = load_lm_model_ckpt(default_model_checkpoint, cache_dir=cache_dir)
cfg_def = OmegaConf.create(pkg_def['xp.cfg'])
cfg_def.memory_saver = {}
cfg_def.memory_saver.enable=False
pkg['best_state']['model']['condition_provider.conditioners.description.output_proj.weight'] = pkg_def['best_state']['condition_provider.conditioners.description.output_proj.weight']
pkg['best_state']['model']['condition_provider.conditioners.description.output_proj.bias'] = pkg_def['best_state']['condition_provider.conditioners.description.output_proj.bias']
lm_model = builders.get_lm_model(cfg_def)
lm_model.load_state_dict(pkg['best_state']['model'])
lm_model.eval()
lm_model.cfg = cfg

In [8]:
lm_model.cfg.dataset.segment_duration = 30

In [9]:
compression_model = load_compression_model(default_model_checkpoint, device=device)
if 'self_wav' in lm_model.condition_provider.conditioners:
    lm_model.condition_provider.conditioners['self_wav'].match_len_on_eval = True
    lm_model.condition_provider.conditioners['self_wav']._use_masking = False

C:\Users\hrayr\anaconda3\envs\MusicGen\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [10]:
model = MusicGen('facebook/musicgen-small', compression_model, lm_model)

In [11]:
model.set_generation_params(duration=30)

In [12]:
model.generation_params

{'use_sampling': True,
 'temp': 1.0,
 'top_k': 250,
 'top_p': 0.0,
 'cfg_coef': 3.0,
 'two_step_cfg': False}

In [13]:
wav = model.generate(["A music that has elements of both Armenian and Armenian folk music, featuring the duduk as an instrument, evoking melancholic and relaxing moods."])

In [14]:
from IPython.display import Audio

In [15]:
import matplotlib.pyplot as plt

In [16]:
Audio(wav[0][0].cpu(), rate=32000)

In [10]:
from scipy.io.wavfile import write

In [9]:
write('output1.wav', 32000, wav[0][0].cpu().numpy())